In [1]:
import graphviz
import pydot
import pywrapfst as pfst
from pywrapfst import compose
# import fst
import nltk
import re
import os
from fst_funcs import analyse_state, get_lex, get_lang, get_rm_dupes, get_rm_blanks, speech_to_text, add_blank_loops, create_frame_acceptor
import numpy as np
import csv
import editdistance
from multiprocessing.dummy import Pool as ThreadPool 

%load_ext line_profiler
%load_ext memory_profiler

In [2]:
LM_expr = "^(.*)\.pru$"
folder_name = "../lang_model"
file_list = [os.path.join(folder_name, fname) for fname in os.listdir(folder_name)]
pruned_models = [re.match(LM_expr, filename).group(1) for filename in file_list if re.match(LM_expr, filename)]
pruned_models

['../lang_model/3-gram-1',
 '../lang_model/3-gram-3',
 '../lang_model/3-gram-0',
 '../lang_model/3-gram-2']

In [3]:
small_lang_model_name = folder_name+"/3-gram-3.pru"
big_lang_model_name = folder_name+"/3-gram-1.pru"

In [27]:
st_w_blank = pfst.SymbolTable().read_text("phoneme-symb-table-with-blank.txt")
st_no_blank = pfst.SymbolTable().read_text("phoneme-symb-table.txt")

In [4]:
big_fst = pfst.Fst.read(big_lang_model_name)
small_fst = pfst.Fst.read(small_lang_model_name)

In [5]:
big_fst.start()

1

In [47]:
big_fst.input_symbols().find("neoplatonists")

KeyError: 'gallbladders'

In [25]:
analyse_state(small_fst, 36)

1 arcs from 36 to 0, e.g: <eps>
1 arcs from 36 to 2, e.g: of


In [29]:
nltk.corpus.cmudict.dict()["thousands"]

[[u'TH', u'AW1', u'Z', u'AH0', u'N', u'D', u'Z'],
 [u'TH', u'AW1', u'Z', u'AH0', u'N', u'Z']]

In [31]:
test = create_frame_acceptor("TH AW Z AH N D Z", st_no_blank)
pfst.compose(test, small_fst)

FstOpError: Operation failed

In [ ]:
def traverse_loops_inner(small_fst, big_fst, quotient_fst, state):
    for arc in fst.arcs(state):
        if arc.nextstate == big_fst.start():
            continue
        add_to_quotient(small_fst, big_fst, quotient_fst, state, arc, arc.nextstate)
        traverse_loops_inner(small_fst, big_fst, quotient_fst, arc.nextstate)
        
def traverse_loops(fst, f):
    quotient_fst = pfst.Fst()
    s = quotient_fst.add_state()
    quotient_fst.set_start(s)
    quotient_fst.set_final(s)    
        
        
def add_to_quotient(small_fst, big_fst, quotient_fst, state, arc, nextstate):
    
    

In [ ]:
traverse_loops(big_fst, big_fst.start())